In [1]:
import numpy as np
import pandas as pd
# Import visualization libraries
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
import seaborn as sns
import os
from astropy import units as u
from astropy.coordinates import SkyCoord
import traceback
from dataclasses import make_dataclass

In [62]:
ybsc_catalog = None
with open('bsc5.dat', 'r') as fi:
    ybsc_catalog = fi.readlines()
ybsc_catalog[0]
print(len(ybsc_catalog))
YBSC_Entry = make_dataclass("YBSC_Entry", [
    ("id", int),
    ("name", str),
    ("constellation", str),
    ("coord", SkyCoord),
    ("glon", float),
    ("glat", float),
    ("gal_coord", SkyCoord),
    ("Vmag", float),
    ("n_Vmag", str),
    ("u_Vmag", str),
    ("spectral_type", str),
    ("spectral_type_full", str),
    ("n_spectral_type", str),
    ("RI", float),
    ("n_RI", str),
    ("BV", float),
    ("u_BV", str),
    ("UB", float),
    ("u_UB", str),
    ("RadVel", float),
    ("RotVel", float),
    ("Parallax", float),
    ("n_Parallax", str),
    ("group", str)
])

# TODO: YBSC also has some other entries for spectral types, should understand what they are
spectral_types = ["O", "B", "A", "F", "G", "K", "M", "D", "S", "C", "W"]

def parse_float(n):
    try:
        return float(n)
    except ValueError:
        return None

def parse_int(n):
    try:
        return int(n)
    except ValueError:
        return None

def parse_str(n):
    try:
        return str(n)
    except ValueError:
        return None

    
entries = []
for i, row in enumerate(ybsc_catalog):
    try:
        star_id = parse_int(row[0:4])
        name = row[4:14]
        constellation = row[11:14].strip()

        coord_str = f"{row[75:77]} {row[77:79]} {row[79:83]} {row[83]}{row[84:86]} {row[86:88]} {row[88:90]}"
        # print(coord_str)
        coord = SkyCoord(coord_str, unit=(u.hourangle, u.deg), frame='icrs')
        
        glon = parse_float(row[90:96])
        glat = parse_float(row[96:102])
        gal_coord = SkyCoord(l=glon, b=glat, frame='galactic', unit=u.degree)

        mag = parse_float(row[102:107])
        n_mag = row[107]
        u_mag = row[108]

        spectral_type_full = row[127:147].strip()
        spectral_type = spectral_type_full[0]
        n_spectral_type = row[147]
        
        if not np.isin(spectral_type, spectral_types):
            # For cases like gG8
            spectral_type = spectral_type_full[1]
            if not np.isin(spectral_type, spectral_types):
                continue
        
        # UBV system
        BV = parse_float(row[109:114])
        u_BV = row[114]
        
        UB = parse_float(row[115:120])
        u_UB = row[120]
        
        RI = parse_float(row[121:126])
        n_RI = row[126]
        
        RadVel = parse_int(row[166:170])
        RotVel = parse_int(row[176:179])
        
        n_parallax = row[160]
        parallax = parse_float(row[161:166])

        group = parse_float(row[205:208])

    
        entries.append(YBSC_Entry(
            id=star_id,
            name=name,
            constellation=constellation,
            coord=coord,
            glon=glon,
            glat=glat,
            gal_coord=gal_coord,
            Vmag=mag,
            n_Vmag=n_mag,
            u_Vmag=u_mag,
            spectral_type=spectral_type,
            spectral_type_full=spectral_type_full,
            n_spectral_type=n_spectral_type,
            BV=BV,
            u_BV=u_BV,
            UB=UB,
            u_UB=u_UB,
            RI=RI,
            n_RI=n_RI,
            RadVel=RadVel,
            RotVel=RotVel,
            Parallax=parallax,
            n_Parallax=n_parallax,
            group = group
        ))
    except ValueError as e:
#         print(traceback.format_exc())
        print(f"Skipping row {i+1} Reason: {e}")
df = pd.DataFrame(entries)
#df['group'].unique()

9110
Skipping row 92 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 95 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 182 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 1057 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 1841 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 2472 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 2496 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 3515 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 3671 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping row 6309 Reason: Cannot parse first argument data "                    " for attribute ra
Skipping r

In [68]:
pleiads_ids = []
with open('ybsc5.notes', 'r') as fi:
    ybsc_notes = fi.readlines()
for row in ybsc_notes:
    if (row[7:13])[0] == 'G':
        if 'Pleiades' in row:
            pleiads_ids.append(int(row[0:5]))
pleiads = df[df['id'].isin(pleiads_ids)]

In [66]:
pleiads

,id,name,constellation,coord,glon,glat,gal_coord,Vmag,n_Vmag,u_Vmag,...,n_RI,BV,u_BV,UB,u_UB,RadVel,RotVel,Parallax,n_Parallax,group
14,15,21Alp And,And,"<SkyCoord (ICRS): (ra, dec) in deg\n (2.097...",111.73,-32.84,"<SkyCoord (Galactic): (l, b) in deg\n (111....",2.06,,,...,,-0.11,,-0.46,,-12.0,56.0,0.032,,None
48,49,,,"<SkyCoord (ICRS): (ra, dec) in deg\n (3.733...",111.92,-39.81,"<SkyCoord (Galactic): (l, b) in deg\n (111....",6.24,,,...,,-0.01,,0.12,,-15.0,NaN,NaN,,None
118,121,13 Cas,Cas,"<SkyCoord (ICRS): (ra, dec) in deg\n (7.855...",120.94,3.73,"<SkyCoord (Galactic): (l, b) in deg\n (120....",6.18,,,...,,-0.10,,-0.47,,-10.0,NaN,NaN,,None
123,126,Bet1Tuc,Tuc,"<SkyCoord (ICRS): (ra, dec) in deg\n (7.886...",306.78,-54.02,"<SkyCoord (Galactic): (l, b) in deg\n (306....",4.37,,,...,,-0.07,,-0.17,,14.0,173.0,0.030,,None
189,193,22Omi Cas,Cas,"<SkyCoord (ICRS): (ra, dec) in deg\n (11.18...",121.78,-14.57,"<SkyCoord (Galactic): (l, b) in deg\n (121....",4.54,,,...,,-0.07,,-0.51,,-17.0,260.0,NaN,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8400,8425,Alp Gru,Gru,"<SkyCoord (ICRS): (ra, dec) in deg\n (332.0...",350.00,-52.47,"<SkyCoord (Galactic): (l, b) in deg\n (350....",1.74,,,...,,-0.13,,-0.47,,12.0,236.0,0.057,,None
8440,8465,21Zet Cep,Cep,"<SkyCoord (ICRS): (ra, dec) in deg\n (332.7...",103.06,1.67,"<SkyCoord (Galactic): (l, b) in deg\n (103....",3.35,,,...,,1.57,,1.71,,-18.0,17.0,0.017,,None
8885,8911,8Kap Psc,Psc,"<SkyCoord (ICRS): (ra, dec) in deg\n (351.7...",83.92,-55.08,"<SkyCoord (Galactic): (l, b) in deg\n (83.9...",4.94,,,...,,0.03,,-0.02,,-3.0,41.0,0.040,,None
8889,8915,69 Peg,Peg,"<SkyCoord (ICRS): (ra, dec) in deg\n (351.9...",99.98,-33.94,"<SkyCoord (Galactic): (l, b) in deg\n (99.9...",5.98,,,...,,-0.06,,-0.17,,-16.0,40.0,NaN,,None
